
# 医学因果推断 · 阶段 1
**目标（2 天）**：只掌握必要的概念，能看懂代码背后的因果逻辑，并用 Python 在 **Lalonde** 数据集（或等价替代）上完成 ATE/ATT 的基础估计与稳健性检验。

> 你可以**从上到下顺序运行**每个单元格。每个章节都包含**简短理论 + 可运行代码**。



## 0. 环境准备

此单元格会安装本 Notebook 需要的包（如果已安装会自动跳过）。
- 数据处理与可视化：`pandas`, `numpy`, `matplotlib`, `seaborn`
- 统计与机器学习：`scikit-learn`, `statsmodels`
- 因果推断：`dowhy`, `econml`（可选，用于进阶）, `causalml`（可选）


In [ ]:

import sys, subprocess, pkgutil

def ensure(pkg):
    if pkg in {m.name for m in pkgutil.iter_modules()}:
        print(f"{pkg} already installed")
    else:
        print(f"Installing {pkg} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

# Core
for p in ["pandas","numpy","matplotlib","seaborn","scikit-learn","statsmodels"]:
    ensure(p)

# Causal packages
for p in ["dowhy","econml","causalml"]:
    try:
        ensure(p)
    except Exception as e:
        print(f"Optional package {p} could not be installed automatically: {e}")
        print("You can try installing it manually later if needed.")



## 1. 最小理论（Minimal Theory）

### 1.1 预测 vs 因果
- **预测**：在当前条件下，结果会是多少？（例如：预测 1 年内是否发生心梗）  
- **因果**：如果我们**干预**（改变）某变量，结果会怎样？（例如：**用药 A 会不会降低心梗风险**）

### 1.2 Pearl 三层次（Causal Hierarchy）
1. **关联**：\(P(Y\mid X)\) —— 观察到 X 时，Y 的分布。  
2. **干预**：\(P(Y\mid do(X))\) —— **强制设置** X 为某值后的 Y。  
3. **反事实**：\(P(Y_{x'} \mid X=x, Y=y)\) —— 实际发生与假设世界对比（“如果没用药，会怎样？”）。

### 1.3 混杂变量（Confounders）与偏倚
- **混杂变量**同时影响治疗（X）与结局（Y），若不调整会导致偏倚。  
- 常见偏倚：选择偏倚、测量偏倚、遗漏变量偏倚。

### 1.4 四类常用方法（本 Notebook 会含代码示例）
- **PSM**：倾向评分匹配  
- **IPTW**：逆概率加权  
- **IV**：工具变量（演示思路）  
- **DR**：双重稳健（AIPW / DR Learner）

> 我们将以 **Lalonde**（就业培训）数据集做演示，它具有与临床场景相似的“治疗/对照 + 混杂”的结构。若在线下载失败，会回退到 **DoWhy 的合成数据**（结构近似）。



## 2. 数据加载：Lalonde（含离线回退）

优先从 **Rdatasets** 在线加载 `MatchIt/lalonde.csv`：  
- `treat`：是否参加项目（类比是否接受治疗）  
- `re78`：1978 年收入（类比结局）  
- 调整变量：`age, educ, black, hispan, married, nodegree, re74, re75`（类比混杂因子）

若网络不可用或下载失败，则**合成一个具备相同结构的模拟数据集**（DoWhy `linear_dataset`）。


In [ ]:

import pandas as pd
import numpy as np

def load_lalonde():
    url = "https://raw.githubusercontent.com/vincentarelbundock/Rdatasets/master/csv/MatchIt/lalonde.csv"
    try:
        df = pd.read_csv(url)
        # Standardize column names
        df = df.rename(columns={"treat":"treat", "re78":"re78"})
        # The Rdatasets version includes an index column 'Unnamed: 0' or first column; drop if present
        if df.columns[0].lower() in {"", "unnamed: 0", "row"}:
            df = df.drop(columns=[df.columns[0]])
        # keep relevant columns if present
        cols = ['treat','re78','age','educ','black','hispan','married','nodegree','re74','re75']
        present = [c for c in cols if c in df.columns]
        df = df[present]
        df['treat'] = df['treat'].astype(int)
        print("Loaded Lalonde from web. Shape:", df.shape)
        source = "lalonde_online"
        return df, source
    except Exception as e:
        print("Failed to load Lalonde online:", e)
        print("Falling back to synthetic dataset with Lalonde-like structure (DoWhy).")
        import dowhy.datasets as dwd
        data = dwd.linear_dataset(
            beta=800,                       # treatment effect magnitude (income scale)
            num_common_causes=8,            # confounders
            num_instruments=0,              # no IV in synthetic baseline
            num_samples=614,                # approx. Lalonde size
            treatment_is_binary=True,
            stddev_treatment_noise=1.0,
            outcome_is_binary=False,
            num_effect_modifiers=0,
            num_discrete_common_causes=5,
            num_continuous_common_causes=3,
            alpha=1.0
        )
        df = data["df"]
        # Align names to mimic Lalonde
        rename_map = {c:f"v{i}" for i,c in enumerate([col for col in df.columns if col not in ["treatment","y"]])}
        for i,(old,new) in enumerate(rename_map.items()):
            df[new] = df.pop(old)
        df = df.rename(columns={"treatment":"treat","y":"re78"})
        print("Generated synthetic Lalonde-like dataset. Shape:", df.shape)
        source = "lalonde_synthetic"
        return df, source

df, data_source = load_lalonde()
df.head()



## 3. 初步探索（EDA）与基线平衡

在因果推断中，EDA 的重点是：
- **治疗组 vs 对照组**在基线特征上的**平衡性**（是否存在系统性差异）  
- 缺失值与异常值处理  
- 连续/分类变量的分布


In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt

print("Shape:", df.shape)
print("\nMissing values per column:")
print(df.isna().sum())

# group counts
print("\nTreatment counts:")
print(df['treat'].value_counts())

# quick numeric summary
display(df.describe(include='all'))


In [ ]:

# 可选：查看治疗组与对照组在关键变量上的差异
numeric_cols = [c for c in df.columns if df[c].dtype != 'O' and c not in ['treat']]
for col in numeric_cols[:6]:  # show a few
    sns.kdeplot(data=df, x=col, hue='treat', common_norm=False)
    plt.title(f"Distribution of {col} by treatment")
    plt.show()



### 3.1 平衡性诊断：标准化差异（SMD）

标准化差异（Standardized Mean Difference, SMD）是衡量组间基线差异的常用指标：  
\( \text{SMD} = \frac{\bar{x}_T - \bar{x}_C}{s} \)，其中 \(s\) 是合并标准差。  
经验规则：\(|\text{SMD}| < 0.1\) 通常认为平衡较好。


In [ ]:

def smd_binary_mask(x, t):
    x_t = x[t==1]; x_c = x[t==0]
    m_t, m_c = np.mean(x_t), np.mean(x_c)
    s = np.sqrt((np.var(x_t, ddof=1)+np.var(x_c, ddof=1))/2)
    return (m_t - m_c)/s if s>0 else 0.0

def smd_table(df, treat_col='treat', exclude=None):
    if exclude is None: exclude=[]
    t = df[treat_col].values
    table = []
    for col in df.columns:
        if col in [treat_col] + exclude: 
            continue
        if pd.api.types.is_numeric_dtype(df[col]):
            val = smd_binary_mask(df[col].values, t)
            table.append((col, val))
    return pd.DataFrame(table, columns=['Variable','SMD']).sort_values('Variable')

smd0 = smd_table(df, treat_col='treat', exclude=['re78'])
display(smd0)



## 4. 倾向评分匹配（PSM）

**思路**：用逻辑回归建模 \(P(T=1\mid X)\) 得到倾向评分（PS），对 PS 相近的个体进行匹配，从而在混杂变量上实现“可比性”。

- 匹配策略：1:1 近邻匹配（不放回），可设置 caliper（半径）限制。  
- 估计量：ATT（对已治疗者的平均处理效应）或 ATE（平均处理效应）。


In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

# Define columns
outcome_col = 're78'
treat_col = 'treat'
covariate_cols = [c for c in df.columns if c not in [outcome_col, treat_col]]

# Drop rows with any na in required columns
work = df[[outcome_col, treat_col] + covariate_cols].dropna().copy()

# 1) Fit propensity model
logit = LogisticRegression(max_iter=1000)
X = work[covariate_cols].select_dtypes(include=[np.number]).values
T = work[treat_col].values
logit.fit(X, T)
ps = logit.predict_proba(X)[:,1]
work['ps'] = ps

# 2) 1:1 nearest-neighbor matching on PS (within caliper optional)
treated_idx = np.where(T==1)[0]
control_idx = np.where(T==0)[0]

nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(ps[control_idx].reshape(-1,1))
distances, indices = nbrs.kneighbors(ps[treated_idx].reshape(-1,1))

# Optional caliper to avoid bad matches
caliper = 0.2 * np.std(ps)  # common heuristic
pairs = []
for i, (d, j) in enumerate(zip(distances.ravel(), indices.ravel())):
    if d <= caliper:
        t_i = treated_idx[i]
        c_j = control_idx[j]
        pairs.append((t_i, c_j))

len(pairs), "matched pairs"


In [ ]:

# 3) Compute ATT from matched pairs
treated_outcomes = work.iloc[[i for i,_ in pairs]][outcome_col].values
control_outcomes = work.iloc[[j for _,j in pairs]][outcome_col].values
att = np.mean(treated_outcomes - control_outcomes)

print(f"PSM ATT (1:1, caliper={caliper:.3f}) = {att:.2f}")


In [ ]:

# 4) Balance after matching (SMD on matched sample)
matched_rows = [i for i,_ in pairs] + [j for _,j in pairs]
matched = work.iloc[matched_rows].copy()

smd_psm = smd_table(matched.rename(columns={treat_col:'treat'}), treat_col='treat', exclude=[outcome_col,'ps'])
display(smd_psm)

print("Before matching: mean |SMD| =", np.mean(np.abs(smd0['SMD'])))
print("After  matching: mean |SMD| =", np.mean(np.abs(smd_psm['SMD'])))



## 5. 倾向评分加权（IPTW）

**思路**：用 \(w=1/PS\)（治疗组）与 \(w=1/(1-PS)\)（对照组）做加权回归，让两组在协变量上“像随机化”一样平衡。

- 这里使用**稳定化权重**减少极端权重的影响。
- 估计 ATE 与 ATT 均可，下面演示 ATE。


In [ ]:

# Stabilized weights
p_t = T.mean()  # marginal P(T=1)
w = np.where(T==1, p_t/ps, (1-p_t)/(1-ps))

work['w'] = w

# Weighted ATE: difference in weighted means
y = work[outcome_col].values
ate_iptw = np.sum(w*(T*y))/np.sum(w*T) - np.sum(w*((1-T)*y))/np.sum(w*(1-T))

print(f"IPTW ATE (stabilized) = {ate_iptw:.2f}")

# Check weight distribution
print("Weight summary:")
display(pd.Series(w).describe(percentiles=[.01,.05,.5,.95,.99]))


In [ ]:

# Balance diagnostics after IPTW (reweight the sample and compute weighted SMD)
def weighted_smd(x, t, w):
    x_t = x[t==1]; x_c = x[t==0]
    w_t = w[t==1]; w_c = w[t==0]
    m_t = np.average(x_t, weights=w_t)
    m_c = np.average(x_c, weights=w_c)
    v_t = np.average((x_t-m_t)**2, weights=w_t)
    v_c = np.average((x_c-m_c)**2, weights=w_c)
    s = np.sqrt((v_t+v_c)/2)
    return (m_t-m_c)/s if s>0 else 0.0

rows = []
for col in covariate_cols:
    if pd.api.types.is_numeric_dtype(work[col]):
        rows.append((col, weighted_smd(work[col].values, T, w)))
smd_iptw = pd.DataFrame(rows, columns=['Variable','Weighted_SMD']).sort_values('Variable')
display(smd_iptw)

print("After IPTW: mean |Weighted SMD| =", np.mean(np.abs(smd_iptw['Weighted_SMD'])))



## 6. 双重稳健（AIPW / DR）

**思路**：同时建模**倾向评分**与**结果模型**，只要二者之一被正确指定，估计仍然无偏。

下面使用最常见的 **AIPW** 实现 ATE：


In [ ]:

from sklearn.ensemble import RandomForestRegressor

# Fit outcome models E[Y|X,T=1] and E[Y|X,T=0]
Xnum = work[covariate_cols].select_dtypes(include=[np.number]).values
y = work[outcome_col].values

m1 = RandomForestRegressor(n_estimators=200, random_state=42)
m0 = RandomForestRegressor(n_estimators=200, random_state=42)
m1.fit(Xnum[T==1], y[T==1])
m0.fit(Xnum[T==0], y[T==0])

mu1 = m1.predict(Xnum)
mu0 = m0.predict(Xnum)

# AIPW pseudo-outcome
# psi = mu1 - mu0 + T*(y - mu1)/ps - (1-T)*(y - mu0)/(1-ps)
psi = (mu1 - mu0) + T*(y - mu1)/ps - (1-T)*(y - mu0)/(1-ps)
ate_aipw = psi.mean()
print(f"AIPW / DR ATE = {ate_aipw:.2f}")



## 7. 使用 DoWhy：因果图（DAG）→ 识别 → 估计 → 反驳/稳健性

我们构造一个简单的 DAG：
- `X=treat`（治疗）影响 `Y=re78`（结局）  
- 混杂变量 `C`（用我们选择的一组基线特征近似）同时影响 X 与 Y  
- 目标：估计 `ATE: E[Y|do(X=1)] - E[Y|do(X=0)]`

> 注意：对于真实医学研究，你应根据临床知识绘制更贴近现实的 DAG。


In [ ]:

from dowhy import CausalModel

# Select covariates available in our data as "common_causes"
common_causes = [c for c in covariate_cols if pd.api.types.is_numeric_dtype(work[c])]

# Build a simple DAG string
# We'll treat all common_causes as parents of both treat and re78
edges = []
for c in common_causes:
    edges.append(f"{c} -> {treat_col}")
    edges.append(f"{c} -> {outcome_col}")
edges.append(f"{treat_col} -> {outcome_col}")
graph_str = "digraph { " + "; ".join(edges) + " }"
print(graph_str[:200] + " ...")

model = CausalModel(
    data=work[[outcome_col, treat_col] + common_causes],
    treatment=treat_col,
    outcome=outcome_col,
    graph=graph_str
)

identified_estimand = model.identify_effect()
print(identified_estimand)


In [ ]:

# Estimate effect via different methods
estimate_psw = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.propensity_score_weighting"
)
print("DoWhy PSW (ATE):", estimate_psw.value)

estimate_reg = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.linear_regression"
)
print("DoWhy Regression (ATE):", estimate_reg.value)

estimate_match = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.propensity_score_matching"
)
print("DoWhy PSM (ATT approx):", estimate_match.value)


In [ ]:

# Refutation tests (robustness checks)
refute_placebo = model.refute_estimate(identified_estimand, estimate_psw, method_name="placebo_treatment_refuter")
print("\nRefutation: placebo treatment ->", refute_placebo)

refute_random_common = model.refute_estimate(identified_estimand, estimate_psw, method_name="random_common_cause")
print("\nRefutation: add random common cause ->", refute_random_common)



## 8. 工具变量（IV）思路（简述）

当存在**未测混杂**时，PSM/IPTW 可能无法消除偏倚。若能找到满足 **相关性** 与 **排他性** 的工具变量 \(Z\)：  
- **相关性**：\(Z\) 影响治疗 \(X\)；  
- **排他性**：\(Z\) 只通过 \(X\) 影响结局 \(Y\)，不通过其它路径。

在医学中，常见候选是**医生的处方倾向**或**地理可及性**。  
实现上常用 **2SLS（两阶段最小二乘）**。此 Notebook 使用的数据没有天然的 IV，后续可以在你的真实研究中尝试。



## 9. 小结与下一步

你已经完成：
- 用 **Lalonde**（或等价合成）数据做了 **PSM / IPTW / AIPW（DR）** 的 ATE/ATT 估计；
- 用 **DoWhy** 完成了 **DAG → 识别 → 估计 → 反驳** 的闭环。

**下一步建议（阶段 2 起）**：
1. 使用 `econml` 的 `DRLearner`、`XLearner` 估计 **CATE/ITE**（个体化治疗效果）；  
2. 在医学数据（如 **MIMIC-IV** 子集）上，完成一个真实世界问题（例如：早期抗生素对 ICU 28 天死亡率的影响）；  
3. 强化 **平衡性诊断**：SMD 分布、Love plot、截断极端权重；  
4. 做 **敏感性分析**（未测混杂强度的 E-value、Rosenbaum bounds 等）。

> 如果你需要，我可以把“阶段 2：Python 实战”的完整 Notebook 再为你生成一份，直接衔接今天的结果。
